# Proyecto DATAMAD

## 1. Carga de datos

## 1.1 Carga de datos de Idealista

In [2]:
import requests

## Esta librería no te servirá en este análisis, pero
## es la librería básica para scrapear páginas más sencillas

from bs4 import BeautifulSoup as bs

## Esta sí. Con ella, sacaremos datos del HTML.

import random

## Para usar números aleatorios. Esta no era tan complicada.
## La usaremos para no usar el mismo tiempo siempre al scrapear.

import time

## Con esta metemos el retardo a la máquina. Las máquinas van rápidas. Yo no.

import pandas as pd
import numpy as np

## EL bread and butter del Data Science. Tablas y operaciones aritméticas.

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

## Esta es la manera para usar selenium normal, pero nosotros tenemos que parecer humanos.
## Por eso, usamos la siguiente librería.

import undetected_chromedriver as uc

## La persona que desarrolló esto es un crack. Configuró todos los proxies y
## demás (digo demás porque no se qué brujería ha usado) para ser indetectable.
## Usaremos este browser.

In [3]:
browser = uc.Chrome()

In [4]:
x=1
ids = []

while True:
    
    url = f'https://www.idealista.com/venta-viviendas/madrid-madrid/pagina-{x}.htm'
    
    browser.get(url)
    
    time.sleep(random.randint(10,12))
    
    try:
        
        browser.find_element("xpath",'//*[@id="didomi-notice-agree-button"]').click()
    except:
        pass
    
    html = browser.page_source
    
    soup = bs(html, 'lxml')
    
    pagina_actual = int(soup.find('main',{'class':'listing-items'}).find('div',{'class':'pagination'}).find('li',{'class':'selected'}).text)
    
    if x == pagina_actual:

        articles = soup.find('main',{'class':'listing-items'}).find_all('article')

    else:
        break
    
    x = x+1
    
    for article in articles:
        
        id_muebles = article.get('data-element-id')
        
        ids.append(id_muebles)
        
        time.sleep(random.randint(1,3))
        print(id_muebles)
    
    ids = [muebles for muebles in ids if muebles is not None]

KeyboardInterrupt: 

In [ ]:
ids_casas = pd.DataFrame(ids)
ids_casas.columns = ['id']

In [ ]:
ids_casas.to_csv('ids_casas.csv',index = False)

In [ ]:
casas = pd.Series()

In [ ]:
def parsear_inmueble(id_inmueble):
    
    print( '\n Casa numero: ' + id_inmueble)
    
    url = "https://www.idealista.com/inmueble/" + id_inmueble + "/"
    
    browser.get(url)

    html = browser.page_source
    
    soup = bs(html, 'lxml')

    titulo = soup.find('span',{'class':'main-info__title-main'}).text
    
    print('\n Titulo: ' + titulo)
    
    localizacion = soup.find('span',{'class':'main-info__title-minor'}).text.split(',')[0]

    print('\n Localizacion: ' + localizacion)
    precio = int(soup.find('span',{'class':'txt-bold'}).text.replace('.',''))

    c1 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-one'})

    caract_basicas = [caract.text.strip() for caract in c1.find_all('li')]
    
    #print('Caracteristicas basicas:' + caract_basicas)

    c2 = soup.find('div',{'class':'details-property'}).find('div',{'class': 'details-property-feature-two'})

    caract_extra = [caract.text.strip() for caract in c2.find_all('li')]
    
    #print('Caracteristicas extras:' + caract_extra)
    
    casas['titulo'] = titulo
    
    casas['localizacion'] = localizacion
    
    casas['precio'] = precio
    
    casas['caracteristicas_basicas'] = caract_basicas
    
    casas['caracteristicas_extras'] = caract_extra
    
    df_casas = pd.DataFrame(casas)
    
    return(df_casas.T)

In [ ]:
df_casas = parsear_inmueble(ids_casas.iloc[0].id)

for i in range(1,len(ids)):
    
    df_casas = pd.concat([df_casas,parsear_inmueble(ids[i])])
    
    time.sleep(random.randint(4,8))

In [ ]:
df_casas.reset_index(drop=True, inplace=True)

In [ ]:
df_casas.to_csv('casas_idealista.csv', index = False, sep = ';', encoding = 'utf-16')

## 1.2 Carga de datos DATAMAD

# 2. Limpieza de datos y coordenadas

In [28]:
import pandas as pd
import re

# Cargar el archivo CSV
df = pd.read_csv("casas_idealista.csv", sep=";", encoding="utf-16")

# Convertir 'caracteristicas_basicas' en una lista de Python, si no está ya en ese formato
df['caracteristicas_basicas'] = df['caracteristicas_basicas'].apply(eval)

# Función para extraer el número de calle
def extraer_numero_calle(titulo):
    # Busca un número en la dirección que esté seguido de coma o espacio
    match = re.search(r',\s?(\d+)', titulo)
    return int(match.group(1)) if match else None

# Aplicar la función para crear la columna 'numero_calle'
df['numero_calle'] = df['titulo'].apply(extraer_numero_calle)

# Función para extraer los datos de 'caracteristicas_basicas'
def extraer_datos(lista):
    datos = {
        'metros_cuadrados': None,
        'habitaciones': None,
        'baños': None,
        'parcela_m2': None,
        'plaza_garaje': None,
        'estado': None,
        'orientacion': None,
        'año_construccion': None,
        'calefaccion': None,
        'planta': None,
        'ascensor': None,
    }

    for item in lista:
        # Solo procesa elementos que contengan características específicas de la vivienda
        if re.search(r'\bm²\b|\bhabitaciones\b|\bbaños\b|garaje|estado|Orientación|Construido en|Calefacción|Planta|ascensor|Parcela', item):
            if re.search(r'\d+\s?m² construidos', item):
                datos['metros_cuadrados'] = int(re.search(r'\d+', item).group())
            elif 'habitaciones' in item:
                datos['habitaciones'] = int(re.search(r'\d+', item).group())
            elif 'baños' in item:
                datos['baños'] = int(re.search(r'\d+', item).group())
            elif 'Parcela de' in item:
                datos['parcela_m2'] = int(re.search(r'\d+', item.replace('.', '')).group())
            elif 'garaje' in item:
                datos['plaza_garaje'] = True
            elif 'Segunda mano' in item or 'para reformar' in item:
                datos['estado'] = item
            elif 'Orientación' in item:
                datos['orientacion'] = item.split(' ')[-1]
            elif 'Construido en' in item:
                datos['año_construccion'] = int(re.search(r'\d+', item).group())
            elif 'Calefacción' in item:
                datos['calefaccion'] = item.split(': ')[-1]
            elif 'Planta' in item:
                datos['planta'] = item
            elif 'ascensor' in item:
                datos['ascensor'] = True

    return datos

# Aplicar la función y expandir los resultados en el DataFrame
datos_df = df['caracteristicas_basicas'].apply(extraer_datos).apply(pd.Series)

# Combinar los datos originales con las nuevas columnas
df = pd.concat([df, datos_df], axis=1)

# Guardar el DataFrame en un nuevo archivo CSV si se desea
df.to_csv('casas_completo.csv', index=False, sep=';')
# Guardar el DataFrame en un nuevo archivo CSV con punto y coma como delimitador



# Mostrar el DataFrame final
print(df.head())


                                              titulo localizacion   precio  \
0          Piso en venta en calle del Duque de Sesto         Goya  1290000   
1          Piso en venta en calle del Duque de Sesto         Goya  1290000   
2                          Ático en venta en Adelfas       Retiro   880000   
3   Chalet pareado en venta en calle Virgen de lo...      Aravaca  2280000   
4   Casa o chalet independiente en venta en calle...   Valdemarín  3500000   

                             caracteristicas_basicas  \
0  [133 m² construidos, 2 habitaciones, 3 baños, ...   
1  [133 m² construidos, 2 habitaciones, 3 baños, ...   
2  [175 m² construidos, 4 habitaciones, 3 baños, ...   
3  [Chalet pareado, 505 m² construidos, 7 habitac...   
4  [Casa o chalet independiente, 749 m² construid...   

                              caracteristicas_extras  numero_calle  \
0   ['Aire acondicionado', 'Consumo:', 'Emisiones:']           NaN   
1   ['Aire acondicionado', 'Consumo:', 'Emisiones:']  

Añadimos las coordenadas a las direcciones utilizando DireccionesVigentes.csv

In [26]:
import pandas as pd
import re

# Cargar los datos
direcciones_df = pd.read_csv("DireccionesVigentes.csv", encoding="latin1", delimiter=";")
casas_df = pd.read_csv("casas_completo.csv", encoding="utf-8", delimiter=";")

# Limpiar y normalizar nombres de calles en DireccionesVigentes
direcciones_df["VIA_NOMBRE"] = direcciones_df["VIA_NOMBRE"].str.upper().str.strip()

# Eliminar duplicados en direcciones_df para que solo haya un valor de coordenadas por calle
direcciones_df = direcciones_df.drop_duplicates(subset=["VIA_NOMBRE"], keep="first")

# Función para extraer y normalizar el nombre de la calle del título
def extraer_nombre_calle(titulo):
    # Buscar el tipo de vía seguido de la última palabra del nombre de la calle
    match = re.search(r'\b(CALLE|AVENIDA|PASEO|PLAZA|RONDA|CAMINO)\s+(?:DE(L|LA)?\s+)?(\w+)$', titulo, re.IGNORECASE)
    if match:
        return match.group(3).upper().strip()  # Solo capturar la última palabra
    return None

# Crear una columna temporal en casas_df para almacenar el nombre de la calle extraído
casas_df["CALLE_EXTRAIDA"] = casas_df["titulo"].apply(extraer_nombre_calle)

# Hacer el merge de ambas bases de datos en función del nombre de la calle
casas_completo = casas_df.merge(direcciones_df[['VIA_NOMBRE', 'LATITUD', 'LONGITUD']],
                                left_on="CALLE_EXTRAIDA", right_on="VIA_NOMBRE", how="left")

# Eliminar la columna temporal y renombrar columnas
casas_completo.drop(columns=["CALLE_EXTRAIDA", "VIA_NOMBRE"], inplace=True)

# Guardar el archivo actualizado
casas_completo.to_csv("casas_completo_actualizado.csv", index=False)
print("Archivo 'casas_completo_actualizado.csv' creado con éxito.")



Archivo 'casas_completo_actualizado.csv' creado con éxito.


In [32]:
casas_con_coordenadas = casas_completo.dropna(subset=['LATITUD', 'LONGITUD'])

# Guardar el archivo de casas con coordenadas
casas_con_coordenadas.to_csv("casas_con_coordenadas.csv", index=False)
print("Archivo 'casas_con_coordenadas.csv' creado con éxito.")

Archivo 'casas_con_coordenadas.csv' creado con éxito.


In [35]:
import pandas as pd
import re

# Definir una función para convertir coordenadas de DMS a decimal
def dms_a_decimal(dms):
    # Separar grados, minutos, segundos y dirección
    degrees, minutes, seconds, direction = re.split("[°'\" ]+", dms.strip())
    decimal = float(degrees) + float(minutes) / 60 + float(seconds) / 3600
    if direction in ['S', 'W']:
        decimal = -decimal
    return decimal

# Cargar los datos desde un CSV
casas_df = pd.read_csv("casas_con_coordenadas.csv", encoding="utf-8", delimiter=",")

# Imprimir los nombres de las columnas para depuración
print("Columnas disponibles en el DataFrame:", casas_df.columns.tolist())

# Convertir las coordenadas de LATITUD y LONGITUD a decimal
# Asegúrate de que los nombres coincidan exactamente con los nombres de columna
if 'LATITUD' in casas_df.columns and 'LONGITUD' in casas_df.columns:
    casas_df['LATITUD'] = casas_df['LATITUD'].apply(dms_a_decimal)
    casas_df['LONGITUD'] = casas_df['LONGITUD'].apply(dms_a_decimal)
else:
    print("No se encontraron las columnas 'LATITUD' y/o 'LONGITUD' en el DataFrame.")

# Guardar el nuevo archivo CSV con las coordenadas transformadas
casas_df.to_csv("casas_con_coordenadas_transformadas.csv", index=False, sep=';')
print("Archivo 'casas_con_coordenadas_transformadas.csv' creado con éxito.")



Columnas disponibles en el DataFrame: ['titulo', 'localizacion', 'precio', 'numero_calle', 'metros_cuadrados', 'habitaciones', 'baños', 'parcela_m2', 'plaza_garaje', 'estado', 'orientacion', 'año_construccion', 'calefaccion', 'planta', 'ascensor', 'LATITUD', 'LONGITUD']
Archivo 'casas_con_coordenadas_transformadas.csv' creado con éxito.


# 3. Aplicacion

In [27]:
import pandas as pd
import streamlit as st

# Cargar la base de datos de pisos
pisos_df = pd.read_csv('G:\Mi unidad\Máster UCM\DATAMAD\DATAMAD\casas_completo.csv', delimiter=';')  # Cambia esta ruta si es necesario

# Títulos y descripción en Streamlit
st.title("Buscador de Pisos en Madrid")
st.markdown("Filtra pisos por criterios de tamaño y precio para encontrar el que mejor se adapte a tus necesidades.")

# Filtros de precio y tamaño
st.sidebar.header("Filtros de búsqueda")

# Precio mínimo y máximo
precio_min = st.sidebar.number_input("Precio mínimo (€)", min_value=0, value=500)
precio_max = st.sidebar.number_input("Precio máximo (€)", min_value=0, value=2000000000)
# Tamaño mínimo y máximo
tamano_min = st.sidebar.number_input("Tamaño mínimo (m²)", min_value=0, value=50)
tamano_max = st.sidebar.number_input("Tamaño máximo (m²)", min_value=0, value=1000000)

# Filtrar pisos por los criterios de precio y tamaño
pisos_filtrados = pisos_df[
    (pisos_df['precio'] >= precio_min) &
    (pisos_df['precio'] <= precio_max) &
    (pisos_df['metros_cuadrados'] >= tamano_min) &
    (pisos_df['metros_cuadrados'] <= tamano_max)
]
# Mostrar resultados filtrados en la aplicación
st.subheader("Pisos encontrados")
st.write(pisos_filtrados[['titulo', 'localizacion', 'precio', 'metros_cuadrados']])


ModuleNotFoundError: No module named 'streamlit'

In [ ]:
import pandas as pd
import streamlit as st
import numpy as np

# Función para calcular la distancia entre dos puntos geográficos
def calcular_distancia(lat1, lon1, lat2, lon2):
    # Convertir grados a radianes
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    # Fórmula de Haversine
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    
    # Radio de la Tierra en metros
    r = 6371000
    return r * c  # Distancia en metros

# Cargar la base de datos de pisos y desfibriladores
pisos_df = pd.read_csv('G:\Mi unidad\Máster UCM\DATAMAD\DATAMAD\casas_con_coordenadas_transformadas.csv', delimiter=';')  # Cambia esta ruta si es necesario
desfibriladores_df = pd.read_csv('G:\Mi unidad\Máster UCM\DATAMAD\DATAMAD\desfibriladores.csv', delimiter=';')  # Cambia esta ruta si es necesario

# Títulos y descripción en Streamlit
st.title("Buscador de Pisos en Madrid")
st.markdown("Filtra pisos por criterios de tamaño y precio para encontrar el que mejor se adapte a tus necesidades.")

# Filtros de precio y tamaño
st.sidebar.header("Filtros de búsqueda")

# Precio mínimo y máximo
precio_min = st.sidebar.number_input("Precio mínimo (€)", min_value=0, value=500)
precio_max = st.sidebar.number_input("Precio máximo (€)", min_value=0, value=2000000000)

# Tamaño mínimo y máximo
tamano_min = st.sidebar.number_input("Tamaño mínimo (m²)", min_value=0, value=50)
tamano_max = st.sidebar.number_input("Tamaño máximo (m²)", min_value=0, value=1000000)

# Filtrar pisos por los criterios de precio y tamaño
pisos_filtrados = pisos_df[
    (pisos_df['precio'] >= precio_min) &
    (pisos_df['precio'] <= precio_max) &
    (pisos_df['metros_cuadrados'] >= tamano_min) &
    (pisos_df['metros_cuadrados'] <= tamano_max)
]

# Filtro de distancia a desfibriladores
st.sidebar.header("Filtrar por cercanía a desfibriladores")
distancia_max = st.sidebar.slider("Distancia máxima (m)", min_value=0, max_value=5000, value=500)  # Cambia el rango según tus necesidades

# Calcular la distancia a desfibriladores
if not pisos_filtrados.empty and not desfibriladores_df.empty:
    # Crear una columna para la distancia mínima a cualquier desfibrilador
    pisos_filtrados['distancia_desfibrilador'] = np.inf  # Inicializa con infinito

    for _, desfibrilador in desfibriladores_df.iterrows():
        lat_d = desfibrilador['direccion_latitud']
        lon_d = desfibrilador['direccion_longitud']
        
        # Calcular distancias
        distancias = pisos_filtrados.apply(lambda x: calcular_distancia(x['LATITUD'], x['LONGITUD'], lat_d, lon_d), axis=1)
        
        # Actualizar la distancia mínima
        pisos_filtrados['distancia_desfibrilador'] = np.minimum(pisos_filtrados['distancia_desfibrilador'], distancias)

    # Filtrar los pisos según la distancia máxima a desfibriladores
    pisos_filtrados = pisos_filtrados[pisos_filtrados['distancia_desfibrilador'] <= distancia_max]

# Mostrar resultados filtrados en la aplicación
st.subheader("Pisos encontrados")
st.write(pisos_filtrados[['titulo', 'localizacion', 'precio', 'metros_cuadrados', 'distancia_desfibrilador']])


ModuleNotFoundError: No module named 'streamlit'

In [41]:
import pandas as pd
import numpy as np

# Función para calcular la distancia entre dos puntos geográficos usando la fórmula de Haversine
def calcular_distancia(lat1, lon1, lat2, lon2):
    # Convertir grados a radianes
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    # Fórmula de Haversine
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Radio de la Tierra en metros
    r = 6371000
    return r * c  # Distancia en metros

# Cargar los datos de casas y desfibriladores
pisos_df = pd.read_csv(r'G:\Mi unidad\Máster UCM\DATAMAD\DATAMAD\casas_con_coordenadas_transformadas.csv', delimiter=';')
desfibriladores_df = pd.read_csv(r'G:\Mi unidad\Máster UCM\DATAMAD\DATAMAD\desfibriladores.csv', delimiter=';', encoding='latin1')

# Reemplazar comas por puntos en las coordenadas de los desfibriladores y convertir a float
desfibriladores_df['direccion_latitud'] = desfibriladores_df['direccion_latitud'].str.replace(',', '.').astype(float)
desfibriladores_df['direccion_longitud'] = desfibriladores_df['direccion_longitud'].str.replace(',', '.').astype(float)

# Mostrar las distancias calculadas
for index, piso in pisos_df.iterrows():
    lat_piso = piso['LATITUD']
    lon_piso = piso['LONGITUD']
    
    print(f"\nDistancias desde la casa en (Lat: {lat_piso}, Lon: {lon_piso}):")
    
    for _, desfibrilador in desfibriladores_df.iterrows():
        lat_desfibrilador = desfibrilador['direccion_latitud']
        lon_desfibrilador = desfibrilador['direccion_longitud']
        
        distancia = calcular_distancia(lat_piso, lon_piso, lat_desfibrilador, lon_desfibrilador)
        print(f" - Desfibrilador en (Lat: {lat_desfibrilador}, Lon: {lon_desfibrilador}): {distancia:.2f} metros")



Distancias desde la casa en (Lat: 40.42081111111111, Lon: -3.685977777777778):
 - Desfibrilador en (Lat: 40.4468295133574, Lon: -3.67717333965445): 2987.54 metros
 - Desfibrilador en (Lat: 40.4475481875495, Lon: -3.67752253090536): 3057.94 metros
 - Desfibrilador en (Lat: 40.4475481875495, Lon: -3.67752253090536): 3057.94 metros
 - Desfibrilador en (Lat: 40.4486621545628, Lon: -3.68265168012371): 3109.66 metros
 - Desfibrilador en (Lat: 40.4324521775606, Lon: -3.69670610885688): 1581.20 metros
 - Desfibrilador en (Lat: 40.4322298601055, Lon: -3.69771772526569): 1612.35 metros
 - Desfibrilador en (Lat: 40.4729333051497, Lon: -3.71526777665801): 6303.45 metros
 - Desfibrilador en (Lat: 40.3923851661772, Lon: -3.6973298755469): 3303.74 metros
 - Desfibrilador en (Lat: 40.395396521516, Lon: -3.70141437367628): 3113.58 metros
 - Desfibrilador en (Lat: 40.391872323738, Lon: -3.69572214908753): 3321.94 metros
 - Desfibrilador en (Lat: 40.3923851661772, Lon: -3.6973298755469): 3303.74 metros


KeyboardInterrupt: 

In [38]:
pip install streamlit

^C
Note: you may need to restart the kernel to use updated packages.


  Using cached streamlit-1.39.0-py2.py3-none-any.whl.metadata (8.5 kB)
  Using cached altair-5.4.1-py3-none-any.whl.metadata (9.4 kB)
  Using cached blinker-1.8.2-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl.metadata (5.3 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached protobuf-5.28.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached rich-13.9.3-py3-none-any.whl.metadata (18 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached GitPython-3.1.43-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached watchdog-5.0.3-py3-none-win_amd64.whl.metadata (41 kB)
  Using cached gitdb-4.0.11-py3-none-any.whl.metadata (1.2 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
 